In [59]:
%%capture
! pip install lightning pandas
 

In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import lightning as L
from torch.utils.data import DataLoader,TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
 

In [61]:
df= pd.read_table("iris.txt",sep=",")
df.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [62]:
df.columns = ["sepal_length",
              "sepal_width",
              "petal_length",
              "petal_width",
              "class"]
df.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [63]:
df.shape

(149, 5)

In [64]:
xi=df[["petal_length",	"petal_width"]]
xi.head()

,petal_length,petal_width
0,1.4,0.2
1,1.3,0.2
2,1.5,0.2
3,1.4,0.2
4,1.7,0.4


In [65]:
yi=df["class"]
yi.head()

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: class, dtype: object

In [66]:
yinum=yi.factorize()[0]
yinum

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [67]:
xtrain,xtest,ytrain,ytest=train_test_split(xi,yinum,test_size=0.25,random_state=40)

In [68]:
print(xtrain.shape,xtest.shape,ytrain.shape,ytest.shape)

(111, 2) (38, 2) (111,) (38,)


In [69]:
ytrainvector=F.one_hot(torch.tensor(ytrain)).type(torch.float32)
ytrainvector[:5]

tensor([[0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [70]:
#NORMALIZATION OF INPUT TRAIN TO RANGE IN BW 0 AND 1. IMPORTANT FOR NEURAL NETWROK

In [71]:
# x=(x-x_min)/)(x_max-x_min)
xmin=xtrain.min()
xmax=xtrain.max()
xtrain=(xtrain-xmin)/(xmax-xmin)
print(xtrain.size,yinum.size)

222 149


In [72]:
xmin=xtest.min()
xmax=xtest.max()
xtest=(xtest-xmin)/(xmax-xmin)


In [73]:
xtraints=torch.tensor(xtrain.values).type(torch.float32)
xtestts=torch.tensor(xtest.values).type(torch.float32)

In [74]:
#Making the data for training

In [75]:
traindata=TensorDataset(xtraints,ytrainvector)
dl=DataLoader(traindata)


Data READY FOR NN. NOW NN MODELLING

This neural netwrok is going to be a fully dense nn

In [76]:
class Net(L.LightningModule):
    def __init__(self):
        super().__init__()
        L.seed_everything(seed=42) # Whatever random number gets generated through L, it remains predictable
        self.input_to_hidden=nn.Linear(in_features=2,out_features=2,bias=True)
        self.hidden_to_output=nn.Linear(in_features=2,out_features=3,bias=True)
        self.loss=nn.MSELoss(reduction="sum")
    def forward(self,x):
        xi= self.input_to_hidden(x)
        yi=F.relu(xi)
        out= self.hidden_to_output(yi)
        return out
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.001)
    def training_step(self, batch, batch_idx):
        # print("training")
        xi,yi=batch
        out=self.forward(xi)
        loss= self.loss(out,yi)
        return loss

In [77]:
model=Net()


Seed set to 42


In [78]:
trainer = L.Trainer(max_epochs=10,enable_progress_bar=False)
trainer.fit(model, train_dataloaders=dl)

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name             | Type    | Params | Mode 
-----------------------------------------------------
0 | input_to_hidden  | Linear  | 6      | train
1 | hidden_to_output | Linear  | 9      | train
2 | loss             | MSELoss | 0      | train
-----------------------------------------------------
15        Trainable params
0         Non-trainable params
15        Total params
0.000     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argumen

In [79]:
pred=model(xtestts)

In [80]:
# pred

In [81]:
argmaxval=torch.argmax(pred,dim=1)
argmaxval

tensor([0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0,
        2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0])

In [82]:
torch.sum(torch.eq(torch.tensor(ytest), argmaxval)) / len(argmaxval)

tensor(0.6579)

In [83]:
path_to_checkpoint = trainer.checkpoint_callback.best_model_path
path_to_checkpoint

'/teamspace/studios/this_studio/signa/chapter_03/lightning_logs/version_19/checkpoints/epoch=9-step=1110.ckpt'

In [84]:
model=Net()
trainer = L.Trainer(max_epochs=100,enable_progress_bar=False)
trainer.fit(model, train_dataloaders=dl)

Seed set to 42
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name             | Type    | Params | Mode 
-----------------------------------------------------
0 | input_to_hidden  | Linear  | 6      | train
1 | hidden_to_output | Linear  | 9      | train
2 | loss             | MSELoss | 0      | train
-----------------------------------------------------
15        Trainable params
0         Non-trainable params
15        Total params
0.000     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode
`Trainer.fit` stopped: `max_epochs=100` reached.


In [85]:
pre=model(xtestts)
argmaxval=torch.argmax(pre,dim=1)


In [86]:
torch.sum(torch.eq(torch.tensor(ytest), argmaxval)) / len(argmaxval)

tensor(0.7368)

In [87]:
path_to_checkpoint = trainer.checkpoint_callback.best_model_path
path_to_checkpoint

'/teamspace/studios/this_studio/signa/chapter_03/lightning_logs/version_20/checkpoints/epoch=99-step=11100.ckpt'